In [1]:
#!pip3 install imblearn
#!pip3 install opencv-python

In [3]:
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import ADASYN

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import glob

# 1. READ-IN DATA TO DF

In [4]:
df = pd.read_csv("labels-map-proj_v3_2.txt", sep=" ", header=None, names=["image", "label"])

# filter all images that end with fv, brt, r90, r180, r270, and fh
df = df[~df["image"].str.contains("fv|brt|r90|r180|r270|fh")]
print(df.shape)
df.head(5)

(10815, 2)


,image,label
0,ESP_013049_0950_RED-0067.jpg,7
7,ESP_019697_2020_RED-0024.jpg,1
14,ESP_015962_1695_RED-0016.jpg,1
21,ESP_013049_0950_RED-0118.jpg,7
28,ESP_015962_1695_RED-0017.jpg,1


In [7]:
df["label"].value_counts()

0    8802
1     794
6     298
3     267
4     250
2     166
7     164
5      74
Name: label, dtype: int64

# 2. DELETE 8k RANDOM IMAGES & SHUFFLE

In [8]:
# delte 7802 random images from the category with label 0
df_us = df.drop(df[df["label"] == 0].sample(7702, random_state=1).index)
df_us["label"].value_counts()

0    1100
1     794
6     298
3     267
4     250
2     166
7     164
5      74
Name: label, dtype: int64

In [9]:
# shuffle the dataframe
df_us = df_us.sample(frac=1, random_state=1).reset_index(drop=True)
df_us.head(5)

,image,label
0,ESP_018720_2655_RED-0035.jpg,2
1,ESP_046991_0950_RED-0024.jpg,7
2,ESP_039350_1915_RED-0186.jpg,1
3,ESP_014156_1865_RED-0023.jpg,3
4,ESP_013049_0950_RED-0088.jpg,7


# 3. READ-IN TO PIXELS

In [10]:
def load_images_labels_from_df(df, folder):
    images = []
    labels = []
    for i in range(len(df)):
        img = cv2.imread(folder+"/"+df.iloc[i][0], cv2.IMREAD_GRAYSCALE)
        if img is not None:
            images.append(img)
            labels.append(df.iloc[i][1])
    return np.array(images), np.array(labels)

In [11]:
X, y = load_images_labels_from_df(df_us, "map-proj-v3_2")
unique, counts = np.unique(y, return_counts=True)
print(unique, counts)
print(X.shape[0])

[0 1 2 3 4 5 6 7] [1100  794  166  267  250   74  298  164]
3113


# 4. NORMALIZE DATA TO 0-1

In [12]:
X_norm = X / 255.0
print(X_norm.min())
print(X_norm.max())

0.0
1.0


# 5. TRAIN-TEST SPLIT

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=0.3, random_state=1)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(2179, 227, 227) (934, 227, 227) (2179,) (934,)


In [35]:
print(np.unique(y_train, return_counts=True))
print(np.unique(y_train, return_counts=True)[1]/y_train.shape[0])

print(np.unique(y_test, return_counts=True))
print(np.unique(y_test, return_counts=True)[1]/y_test.shape[0])

(array([0, 1, 2, 3, 4, 5, 6, 7]), array([774, 539, 115, 176, 183,  52, 222, 118]))
[0.35520881 0.24736117 0.0527765  0.080771   0.08398348 0.02386416
 0.1018816  0.05415328]
(array([0, 1, 2, 3, 4, 5, 6, 7]), array([326, 255,  51,  91,  67,  22,  76,  46]))
[0.3490364  0.27301927 0.05460385 0.09743041 0.07173448 0.0235546
 0.08137045 0.04925054]


In [33]:
max_images = np.unique(y_train, return_counts=True)[1].max()
n_images = np.unique(y_train, return_counts=True)[1]
classes = np.unique(y_train, return_counts=True)[0]
diff = =

for i in classes:
    if n_images[i] < max_images:

    
    print(n_images[i])


774
539
115
176
183
52
222
118


# 6. IMBALANCE HANDLING

# 6.1 ADASYN

In [20]:
X_reshaped = X_train.flatten().reshape(X_train.shape[0], 51529)
print(X_train.shape)
print(X_reshaped.shape)

(2179, 227, 227)
(2179, 51529)


In [21]:
adasyn = ADASYN(sampling_strategy="not majority", random_state=1)
X_train_adasyn, y_train_adasyn = adasyn.fit_resample(X_reshaped, y_train)

In [22]:
unique, counts = np.unique(y_train, return_counts=True)
unique_a, counts_a = np.unique(y_train_adasyn, return_counts=True)
print(X_train_adasyn.shape)
print(unique, counts)
print(unique_a, counts_a)

(6226, 51529)
[0 1 2 3 4 5 6 7] [774 539 115 176 183  52 222 118]
[0 1 2 3 4 5 6 7] [774 824 763 772 789 762 760 782]


In [23]:
# sort the array counts_a
np.sort(counts_a)
np.average(counts_a)

778.25

In [24]:
X_t_A = X_train_adasyn.reshape(X_train_adasyn.shape[0], 227, 227)
y_t_A = y_train_adasyn
print(X_t_A.shape)
print(y_t_A.shape)

(6226, 227, 227)
(6226,)


# 6.2 OVERSAMPLING

# 6. AUGMENTATION
1. Rotate 90
2. Rotate 180
3. Rotate 270
4. Flip Horizontally
5. Flip Vertically
6. Zoom
7. (Random Brightness)?

In [ ]:
def zoom_at(img, zoom=1.0):
    h, w, = [ zoom * i for i in img.shape ]
    cx, cy = w/2, h/2
    img = cv2.resize( img, (0, 0), fx=zoom, fy=zoom)
    img = img[int(round(cy - h/zoom * .5)) : int(round(cy + h/zoom * .5)),
              int(round(cx - w/zoom * .5)) : int(round(cx + w/zoom * .5))]
    return img

# function that rotates an image by 90, 180, and 270 degrees and flips it horizontally and vertically and zooms in on the image and adjusts the brightness randomly
def augment_image_zoom(img):
    img_90 = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE) # rotate the image by 90 degrees clockwise
    img_180 = cv2.rotate(img, cv2.ROTATE_180) # rotate the image by 180 degrees
    img_270 = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE) # rotate the image by 270 degrees clockwise
    img_fh = cv2.flip(img, 1) # flip the image horizontally
    img_fv = cv2.flip(img, 0) # flip the image vertically
    img_zoom = zoom_at(img, 1.2) # zoom in on the image
    img_brt = cv2.convertScaleAbs(img, beta=0.8) # adjust the brightness randomly
    return img, img_90, img_180, img_270, img_fh, img_fv, img_zoom, img_brt

In [ ]:
# write a function that creates a copy of that image and adds it to the X_t_A array
def add_image(X_t_A, y_t_A, img, label):
    X_t_A = np.append(X_t_A, img)
    y_t_A = np.append(y_t_A, label)
    return X_t_A, y_t_A

# 7. Apply Networks

# 7.1 AlexNet

In [ ]:
model = keras.models.Sequential([
    keras.layers.Conv2D(96, (11, 11), strides=(4, 4), padding='valid', activation='relu') # layer that convolves the input image with 96 different filters that are 11x11 pixels in size
    keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='valid') # layer that downsamples the input representation by taking the maximum value over the window defined by pool_size for each dimension along the features axis
    keras.layers.Conv2D(256, (5, 5), strides=(1, 1), padding='same', activation='relu') # layer that convolves the input image with 256 different filters that are 5x5 pixels in size
    keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='valid') # layer that downsamples the input representation by taking the maximum value over the window defined by pool_size for each dimension along the features axis
    keras.layers.Conv2D(384, (3, 3), strides=(1, 1), padding='same', activation='relu') # layer that convolves the input image with 384 different filters that are 3x3 pixels in size
    keras.layers.Conv2D(384, (3, 3), strides=(1, 1), padding='same', activation='relu') # layer that convolves the input image with 384 different filters that are 3x3 pixels in size
    keras.layers.Conv2D(384, (3, 3), strides=(1, 1), padding='same', activation='relu') # layer that convolves the input image with 384 different filters that are 3x3 pixels in size
    keras.layers.Dense(4096, activation='relu') # layer with 4096 neurons that uses the ReLU activation function and is fully connected
    keras.layers.Dropout(0.5), # layer that randomly sets input units to 0 with a 50% frequency at each step during training time, which helps prevent overfitting
    keras.layers.Dense(4096, activation='relu') # layer with 4096 neurons that uses the ReLU activation function and is fully connected
    keras.layers.Dropout(0.5), # layer that randomly sets input units to 0 with a 50% frequency at each step during training time, which helps prevent overfitting
    keras.layers.Dense(8, activation='softmax') # layer with 8 neurons that uses the softmax activation function to output probabilities for each class
])

- Hyperparameter Optimization